<a href="https://colab.research.google.com/github/DeviManognya/Assignment1/blob/main/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch torchvision scikit-learn matplotlib pandas numpy stanza

In [ ]:
import zipfile
import os

In [ ]:
stanza.download('en')

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Downloaded file to /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources


In [ ]:
def extract_zip(zip_path, extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

In [ ]:
!wget http://mlg.ucd.ie/files/datasets/bbc-fulltext.zip

--2024-03-02 10:55:33--  http://mlg.ucd.ie/files/datasets/bbc-fulltext.zip
Resolving mlg.ucd.ie (mlg.ucd.ie)... 137.43.93.132
Connecting to mlg.ucd.ie (mlg.ucd.ie)|137.43.93.132|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2874079 (2.7M) [application/zip]
Saving to: ‘bbc-fulltext.zip’

bbc-fulltext.zip    100%[===================>]   2.74M  4.00MB/s    in 0.7s    

2024-03-02 10:55:34 (4.00 MB/s) - ‘bbc-fulltext.zip’ saved [2874079/2874079]



In [ ]:
extract_zip("bbc-fulltext.zip", "./")

In [ ]:
os.remove("bbc-fulltext.zip")

In [ ]:
def create_dataset():
    X = []
    y = []
    # Assuming the BBC dataset is organized in folders by category (BUSINESS, ENTERTAINMENT, POLITICS, SPORT, TECH)
    categories = ['business', 'entertainment', 'politics', 'sport', 'tech']
    for category in categories:
        folder_path = os.path.join('bbc', category)
        file_names = os.listdir(folder_path)
        for file_name in file_names:
            file_path = os.path.join(folder_path, file_name)
            with open(file_path, 'r', encoding='utf-8') as file:
                text = file.read()
                sentences = text.split('.')  # Split text into sentences
                X.extend(sentences)
                y.extend([category] * len(sentences))  # Assign label to each sentence
    return X, y


In [ ]:
X, y = create_dataset()

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

le = LabelEncoder()
y = to_categorical(le.fit_transform(y))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_dev, y_train, y_dev = train_test_split(X, y, test_size=0.2, random_state=42)
X_dev, X_test, y_dev, y_test = train_test_split(X_dev, y_dev, test_size=0.5, random_state=42)

In [ ]:
print("Training set lengths - X_train: {}, y_train: {}".format(len(X_train), len(y_train)))
print("Development set lengths - X_dev: {}, y_dev: {}".format(len(X_dev), len(y_dev)))
print("Test set lengths - X_test: {}, y_test: {}".format(len(X_test), len(y_test)))

Training set lengths - X_train: 36753, y_train: 36753
Development set lengths - X_dev: 4594, y_dev: 4594
Test set lengths - X_test: 4595, y_test: 4595


In [ ]:
import tensorflow_hub as hub
embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
X_train = embed(X_train)
X_dev = embed(X_dev)
X_test = embed(X_test)

In [ ]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
model = Sequential()
model.add(Dense(units=32, activation="relu"))
model.add(Dense(units=64, activation="relu"))
model.add(Dense(units=5, activation="softmax"))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(x=X_train, y=y_train,
          epochs=10,
          validation_data=(X_dev, y_dev),
          batch_size=32,
          verbose=1)

Epoch 1/10
1149/1149 [==============================] - 4s 3ms/step - loss: 0.6594 - accuracy: 0.7604 - val_loss: 0.5661 - val_accuracy: 0.7923
Epoch 2/10
1149/1149 [==============================] - 3s 2ms/step - loss: 0.5489 - accuracy: 0.7938 - val_loss: 0.5531 - val_accuracy: 0.7947
Epoch 3/10
1149/1149 [==============================] - 3s 2ms/step - loss: 0.5322 - accuracy: 0.7987 - val_loss: 0.5467 - val_accuracy: 0.7976
Epoch 4/10
1149/1149 [==============================] - 4s 3ms/step - loss: 0.5162 - accuracy: 0.8037 - val_loss: 0.5440 - val_accuracy: 0.8015
Epoch 5/10
1149/1149 [==============================] - 3s 3ms/step - loss: 0.5012 - accuracy: 0.8085 - val_loss: 0.5411 - val_accuracy: 0.7987
Epoch 6/10
1149/1149 [==============================] - 3s 2ms/step - loss: 0.4858 - accuracy: 0.8150 - val_loss: 0.5438 - val_accuracy: 0.7926
Epoch 7/10
1149/1149 [==============================] - 3s 2ms/step - loss: 0.4696 - accuracy: 0.8221 - val_loss: 0.5422 - val_accuracy:

In [ ]:
report = classification_report(y_test, predictions, output_dict=True)

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

In [ ]:
predictions = [str(p) for p in predictions]
y_test = [str(y) for y in y_test]

In [ ]:
report = classification_report(y_test, predictions, output_dict=True)

In [ ]:
import numpy as np
from sklearn.metrics import classification_report


In [ ]:
predictions = [str(p) for p in predictions]
y_test = [str(y) for y in y_test]


In [ ]:
report = classification_report(y_test, predictions, output_dict=True, zero_division='warn')

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
print("\t\tprecision\trecall\tf1-score")
for category, metrics in report.items():
    if category in ['accuracy', 'macro avg', 'weighted avg']:
        print(category, end="\t")
        print(round(metrics['precision'], 2) if isinstance(metrics, dict) else round(metrics, 2), end="\t\t")
        print(round(metrics['recall'], 2) if isinstance(metrics, dict) else "", end="\t\t")
        print(round(metrics['f1-score'], 2) if isinstance(metrics, dict) else "")
    else:
        print("{}\t{}".format(category, "\t".join([str(round(metrics[metric], 2)) for metric in ['precision', 'recall', 'f1-score']])))

		precision	recall	f1-score
business	1.0	0.25	0.39
entertainment	0.0	0.0	0.0
politics	0.0	0.0	0.0
sport	0.0	0.0	0.0
tech	0.0	0.0	0.0
accuracy	0.25				
macro avg	0.2		0.05		0.08
weighted avg	1.0		0.25		0.39
